In [1]:
import pandas as pd

In [2]:
df = pd.read_json(path_or_buf='gg2013.json')
# df = pd.read_json(path_or_buf='gg2015.json')

In [3]:
data = df['text']

In [4]:
# sample data if necessary
sample_size = 200000
if len(df) > sample_size:
    data = data.sample(n=sample_size)
len(data)

174643

In [26]:
import json
with open('autograder/gg2013answers.json') as file:
    ground_truth = json.load(file)
file.closed

True

In [42]:
true_award_winners = ground_truth['award_data']
awards_old = [item[0] for item in true_award_winners.items()]

In [43]:
presenters_old = [true_award_winners[item[0]]['presenters'] for item in true_award_winners.items()]
presenters_old

[['robert pattinson', 'amanda seyfried'],
 ['halle berry'],
 ['aziz ansari', 'jason bateman'],
 ['arnold schwarzenegger', 'sylvester stallone'],
 ['bradley cooper', 'kate hudson'],
 ['dennis quaid', 'kerry washington'],
 ['dustin hoffman'],
 ['will ferrell', 'kristen wiig'],
 ['don cheadle', 'eva longoria'],
 ['jennifer lopez', 'jason statham'],
 ['nathan fillion', 'lea michele'],
 ['george clooney'],
 ['robert downey, jr.'],
 ['jennifer garner'],
 ['julia roberts'],
 ['kristen bell', 'john krasinski'],
 ['megan fox', 'jonah hill'],
 ['salma hayek', 'paul rudd'],
 ['jessica alba', 'kiefer sutherland'],
 ['don cheadle', 'eva longoria'],
 ['sacha baron cohen'],
 ['jennifer lopez', 'jason statham'],
 ['george clooney'],
 ['jimmy fallon', 'jay leno'],
 ['salma hayek', 'paul rudd'],
 ['lucy liu', 'debra messing']]

In [44]:
# hard-coded awards 2013-2015
awards_old

['best screenplay - motion picture',
 'best director - motion picture',
 'best performance by an actress in a television series - comedy or musical',
 'best foreign language film',
 'best performance by an actor in a supporting role in a motion picture',
 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television',
 'best motion picture - comedy or musical',
 'best performance by an actress in a motion picture - comedy or musical',
 'best mini-series or motion picture made for television',
 'best original score - motion picture',
 'best performance by an actress in a television series - drama',
 'best performance by an actress in a motion picture - drama',
 'cecil b. demille award',
 'best performance by an actor in a motion picture - comedy or musical',
 'best motion picture - drama',
 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television',
 'best performance by an actr

In [45]:
# hard-coded awards for 2018-2019
awards = ['Best Motion Picture - Drama',
         'Best Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Motion Picture - Drama',
         'Best Performance by an Actor in a Motion Picture - Drama',
         'Best Performance by an Actress in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actor in a Motion Picture - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in any Motion Picture',
         'Best Performance by an Actor in a Supporting Role in any Motion Picture',
         'Best Director - Motion Picture',
         'Best Screenplay - Motion Picture',
         'Best Motion Picture - Animated',
         'Best Motion Picture - Foreign Language',
         'Best Original Score - Motion Picture',
         'Best Original Song - Motion Picture',
         'Best Television Series - Drama',
         'Best Television Series - Musical or Comedy',
         'Best Television Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actress in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television',
         'Best Performance by an Actress In A Television Series - Drama',
         'Best Performance by an Actor In A Television Series - Drama',
         'Best Performance by an Actress in a Television Series - Musical or Comedy',
         'Best Performance by an Actor in a Television Series - Musical or Comedy',
         'Best Performance by an Actress in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television',
         'Cecil B. deMille Award']

In [52]:
# use awards_old and presenters_old
awards = awards_old
presenters = presenters_old

In [47]:
import re

In [63]:
def reduce(line):
    # test
    # replace "television" to "tv"
    pattern = r'\btelevision\b'
    line = re.sub(pattern, 'tv', line.lower())

    # remove words "best", "performance", "motion", "picture", "limited", "language", "role", in", "a", "an", "any", "made", "for", "by", "b.", "award", and all punctuations
    pattern = r'\bbest\b|\bperformance\b|\bmotion\b|\bpicture\b|\blimited\b|\blanguage\b|\brole\b|\bin\b|\ba\b|\ban\b|\bany\b|\bmade\b|\bfor\b|\bby\b|\bb\b|\baward\b|[^\w\s]'
    return re.sub(pattern, ' ', line.lower()).split()

awards_reduced = [sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]

# test
import pprint
pprint.pprint(awards_reduced)

[['screenplay'],
 ['director'],
 ['actress', 'tv', 'series', 'comedy', 'or', 'musical'],
 ['foreign', 'film'],
 ['actor', 'supporting'],
 ['actress', 'supporting', 'series', 'mini', 'or', 'tv'],
 ['comedy', 'or', 'musical'],
 ['actress', 'comedy', 'or', 'musical'],
 ['mini', 'series', 'or', 'tv'],
 ['original', 'score'],
 ['actress', 'tv', 'series', 'drama'],
 ['actress', 'drama'],
 ['cecil', 'demille'],
 ['actor', 'comedy', 'or', 'musical'],
 ['drama'],
 ['actor', 'supporting', 'series', 'mini', 'or', 'tv'],
 ['actress', 'supporting'],
 ['tv', 'series', 'drama'],
 ['actor', 'mini', 'series', 'or', 'tv'],
 ['actress', 'mini', 'series', 'or', 'tv'],
 ['animated', 'feature', 'film'],
 ['original', 'song'],
 ['actor', 'drama'],
 ['tv', 'series', 'comedy', 'or', 'musical'],
 ['actor', 'tv', 'series', 'drama'],
 ['actor', 'tv', 'series', 'comedy', 'or', 'musical']]


In [64]:
def generate_award_num_keywords_map(awards):
    awards_reduced = []
    award_num_keywords_map = {}
    for i, award in enumerate([sorted(set(reduce(award)), key=lambda word: reduce(award).index(word)) for award in awards]):
        if 'or' in award:
            # find number of words from "or" to the end of award name
            num_words = len(award) - 1 - award.index('or')
            award.remove('or')
            # do not count words from "or" to the end of award name
            award_num_keywords_map[i] = len(award) - num_words 
        else:
            award_num_keywords_map[i] = len(award)
        awards_reduced.append(award)
    return award_num_keywords_map, awards_reduced

In [65]:
award_num_keywords_map, awards_reduced = generate_award_num_keywords_map(awards)

pairs = sorted(award_num_keywords_map.items(), key=lambda pair: pair[0])

In [66]:
for pair in pairs:
    print(awards_reduced[pair[0]])
    # (index, num_keywords_to_match)
    print(pair)
    if pair[1] < len(awards_reduced[pair[0]]):
        print('***') # for the convenience to check awards without primary target words, e.g. ['mini', 'series', 'tv']
    print

['screenplay']
(0, 1)
['director']
(1, 1)
['actress', 'tv', 'series', 'comedy', 'musical']
(2, 4)
***
['foreign', 'film']
(3, 2)
['actor', 'supporting']
(4, 2)
['actress', 'supporting', 'series', 'mini', 'tv']
(5, 4)
***
['comedy', 'musical']
(6, 1)
***
['actress', 'comedy', 'musical']
(7, 2)
***
['mini', 'series', 'tv']
(8, 2)
***
['original', 'score']
(9, 2)
['actress', 'tv', 'series', 'drama']
(10, 4)
['actress', 'drama']
(11, 2)
['cecil', 'demille']
(12, 2)
['actor', 'comedy', 'musical']
(13, 2)
***
['drama']
(14, 1)
['actor', 'supporting', 'series', 'mini', 'tv']
(15, 4)
***
['actress', 'supporting']
(16, 2)
['tv', 'series', 'drama']
(17, 3)
['actor', 'mini', 'series', 'tv']
(18, 3)
***
['actress', 'mini', 'series', 'tv']
(19, 3)
***
['animated', 'feature', 'film']
(20, 3)
['original', 'song']
(21, 2)
['actor', 'drama']
(22, 2)
['tv', 'series', 'comedy', 'musical']
(23, 3)
***
['actor', 'tv', 'series', 'drama']
(24, 4)
['actor', 'tv', 'series', 'comedy', 'musical']
(25, 4)
***


In [14]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
    return re.sub(pattern, ' ', line)

In [15]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [16]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
    return re.sub(pattern, ' ', line)

In [17]:
def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    return re.sub(pattern, ' ', line)

In [18]:
def cleanse(line):
    # replace everything to ' ' except alphanumeric character, whitespace, apostrophe, hashtag
    return re.sub(r'[^\w\s\'#]', ' ', line)

In [20]:
cleansed_data = []
for tweet in data:
    line = remove_retweet_prefix(tweet)
    line = remove_hashtag(line)
    line = remove_at(line)
    line = remove_url(line)
    line = cleanse(line)
    cleansed_data.append(line)

In [21]:
import spacy
# python3 -m spacy download en
nlp = spacy.load('en')

In [22]:
def identify_entities(text):
    entities = list(nlp(text).ents)
    tags = {}
    for entity in entities:
        if entity not in tags:
            tags[' '.join(t.orth_ for t in entity).strip()]=[entity.label_]
    return tags

In [69]:
# top retweets (each as an entity)
sorted(retweets_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes: ', 135540),
 ('eonline: ', 113549),
 ('PerezHilton: ', 90513),
 ('TheEllenShow: ', 66729),
 ('EmWatson: ', 45667),
 ('VanityFair: ', 32168),
 ('nbcsnl: ', 29035),
 ('CNNshowbiz: ', 27278),
 ('CiudadBizarra: ', 20028),
 ('BuzzFeed: ', 19173),
 ('EW: ', 18948),
 ('nbc: ', 18310),
 ('vulture: ', 17555),
 ('piersmorgan: ', 17336),
 ('MARLONLWAYANS: ', 15039),
 ('HuffingtonPost: ', 14371),
 ('buckhollywood: ', 13901),
 ('MarilynMonroeES: ', 13203),
 ('TVGuide: ', 12554),
 ('DavidSpade: ', 12407),
 ('THR: ', 12322),
 ('MTVNews: ', 12015),
 ('PimpBillClinton: ', 11139),
 ('cinema21: ', 10856),
 ('washingtonpost: ', 10790),
 ('ninagarcia: ', 10153),
 ('Cosmopolitan: ', 9400),
 ('peopleenespanol: ', 8599),
 ('RichardCrouse: ', 8530),
 ('prodigalsam: ', 8426),
 ('peoplemag: ', 8336),
 ('kumailn: ', 8327),
 ('DannyZuker: ', 8264),
 ('DamienFahey: ', 7884),
 ('jianghomeshi: ', 7869),
 ('HuffPostWomen: ', 7830),
 ('girlsHBO: ', 7684),
 ('MHarrisPerry: ', 7601),
 ('rogergzz: ', 7536

In [70]:
# top hashtags
sorted(hashtag_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('GoldenGlobes', 4337876),
 ('goldenglobes', 1305145),
 ('Argo', 32266),
 ('GetGlue', 30489),
 ('Homeland', 27745),
 ('redcarpet', 26868),
 ('GoldenGlobe', 22321),
 ('Goldenglobes', 22020),
 ('JodieFoster', 18830),
 ('killingit', 17759),
 ('Girls', 17507),
 ('GOLDENGLOBES', 16562),
 ('AlfombraRojaE', 15224),
 ('Skyfall', 14360),
 ('GlobosdeOro', 13852),
 ('RedCarpet', 13851),
 ('Lincoln', 13525),
 ('LesMis', 13393),
 ('GIRLS', 12822),
 ('Adele', 11519),
 ('LesMiserables', 10894),
 ('JenniferLawrence', 10119),
 ('GG2013', 10033),
 ('homeland', 10021),
 ('GoldenGlobes2013', 9842),
 ('ERedCarpet', 8910),
 ('TinaFey', 8816),
 ('eredcarpet', 8246),
 ('AmyPoehler', 8148),
 ('jodiefoster', 7922),
 ('DowntonAbbey', 7662),
 ('AnneHathaway', 7593),
 ('DjangoUnchained', 7311),
 ('girls', 7310),
 ('Oscars', 6837),
 ('Django', 6279),
 ('LoveThoseLadies', 6194),
 ('BenAffleck', 5925),
 ('GameChange', 5838),
 ('LOVEHER', 5833),
 ('Globes', 5814),
 ('BillClinton', 5672),
 ('goldenGlobes', 5617),
 ('f

In [71]:
# top @
sorted(at_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:50]

[('goldenglobes', 124271),
 ('OfficialAdele', 44704),
 ('lenadunham', 38250),
 ('GoldenGlobes', 34171),
 ('BenAffleck', 20195),
 ('RealHughJackman', 14696),
 ('SHO_Homeland', 14217),
 ('PerezHilton', 14177),
 ('girlsHBO', 13586),
 ('SofiaVergara', 12006),
 ('eonline', 11378),
 ('taylorswift13', 10619),
 ('TNTLA', 10069),
 ('JLo', 9312),
 ('PaulEpworth', 8880),
 ('GirlsHBO', 7060),
 ('msleamichele', 6621),
 ('SelenaGomez', 6515),
 ('VanessuHudgens', 6475),
 ('steph_hart', 6225),
 ('LenaDunham', 6125),
 ('RyanGosling', 5230),
 ('kerrywashington', 4965),
 ('PixarBrave', 4739),
 ('Chanel', 4467),
 ('nbc', 4440),
 ('LesMiserables', 4378),
 ('NathanFillion', 4170),
 ('KevalBaxi', 3686),
 ('Burberry', 3435),
 ('Lewis_Damian', 3429),
 ('jessicaalba', 3326),
 ('CHANEL', 3309),
 ('tomandlorenzo', 3116),
 ('iamdoncheadle', 3023),
 ('nbcsnl', 3005),
 ('piersmorgan', 2666),
 ('CiudadBizarra', 2620),
 ('CNNshowbiz', 2498),
 ('BuzzFeed', 2496),
 ('LeoDiCaprio', 2470),
 ('EvaLongoria', 2414),
 ('aziza

In [60]:
from fuzzywuzzy import fuzz

def remove_goldeb_globes(top_results, entity_freq_dict):
    golden_globes = [name for name in [pair[0] for pair in top_results] if fuzz.ratio(name.lower(), 'golden globes') > 60]
    for name in golden_globes:
        if name in entity_freq_dict:
            del entity_freq_dict[name]
    return entity_freq_dict


# we also consider dropping all lower cases examples or examples that contain digit(s), which are not names
def filter_names(pair_list, entity_freq_dict):
    filtered_results = []
    for pair in pair_list:
        string = ''.join(pair[0].split())
        if not all(char.islower() for char in string) and not any(char.isdigit() for char in string):
            filtered_results.append(pair)
        else:
            if pair[0] in entity_freq_dict:
                del entity_freq_dict[pair[0]]
    return filtered_results, entity_freq_dict


def merge_names(top_results, entity_freq_dict):
    names = [pair[0] for pair in top_results]
    names_clusters = []
    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)
        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)
        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    # find names clusters that should merge
    # ['Amy Poehler', 'Amy', 'Amy Poelher']
    # ['Tina', 'Tina Fey']

    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)

    # weighted frequency of an entity is defined by its frequency multiplied by its string length
    def weighted_freq(element):
        return entity_freq_dict[element] * len(element)

    e = entity_freq_dict.copy()
    for cluster in names_clusters_reduced:
        # select the entity name with highest weighted frequency
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in e and selected_entity_name in e:
                e[selected_entity_name] += e[name]
                del e[name]

    top_10 = sorted(e.items(), key=lambda pair: pair[1], reverse=True)[:10]
    return top_10

In [25]:
%%time
# test
num = 0
count = 0
pattern = re.compile(r'\bpresenter|\bpresent', re.IGNORECASE)
for line in cleansed_data:
    match = re.findall(pattern, line.lower())                  
    if match:
        tags = identify_entities(line)
        count += 1
        # check the first 10 cases
        if count < 10:
            print(num)
            print(match)
            print(line)
            print(tags.keys())
            print()
    num += 1
    
print(count)

48
['present']
Definetely Kate's top designers are present in the   tonight of course thanks to the  
dict_keys(['Kate', 'tonight'])

465
['present']
    comentando los     mejores presentadores no hay  
dict_keys([])

1176
['present']
     comentando los     mejores presentadores no hay  
dict_keys(['', 'los'])

1561
['presenter']
 What's your verdict on   presenter Giuliana Rancic's Oriett Domenech dress     
dict_keys(["Giuliana Rancic 's", 'Oriett Domenech'])

1976
['presenter']
THISSSSSSSSSS RT   It's appalling that Kerry Washington is there only as a presenter and not as a nominee     
dict_keys(['THISSSSSSSSSS RT', 'Kerry Washington'])

3247
['presenter']
Jay Leno a presenter      
dict_keys(['Jay Leno', ''])

3451
['presenter']
 Presenter  amp  Oscar Award winner Halle Berry is showstopping in a printed gown from Atelier Versace Fall 2012 at the Golden Globes right now 
dict_keys(['Presenter', 'Oscar Award', 'Halle Berry', 'Atelier Versace', 'Fall 2012', 'the Golden Globes'])



In [58]:
def check_target_words(awards_reduced):
    target_words = ['actor', 'actress', 'director', 'score', 'song']
    for word in target_words:
        if word in awards_reduced:
            return word
    return False

check_target_words(awards_reduced[12])

False

In [69]:
def find_award_presenter(awards_reduced, award_index, verbose=False):
    print(award_index)
    print('Predicting for:', awards[award_index])
    print('true presenter:', presenters[award_index])
    award = awards_reduced[award_index]
    print('award name reduced:', award)
    num_keywords_to_match = award_num_keywords_map[award_index]
    print("num keywords to match:", num_keywords_to_match)
    
    # add word boundary '\b' to prevent grabbing examples like "showing" and "wonder"    
    pattern = re.compile(r'\bpresenter|\bpresent', re.IGNORECASE)
        
    entity_dict = {}
    
    num = 0
    flag = 0
    target_word = check_target_words(awards_reduced[award_index])
#     if target_word:
#         print("matching target word '{0}'".format(target_word))
    
    # if len(award) != num_keywords_to_match (awards that have 'or' options)
    # and len(award) != 2 (for the case of ['musical', 'comedy']), 
    # make sure the target word in award is matched
#     if (len(award) != num_keywords_to_match and len(award) != 2) or target_word:
    if (len(award) != num_keywords_to_match and len(award) != 2):
        flag = 1
        # find target word pattern to match for sure
        if target_word:
            target_word_pattern = re.compile(r'\b{0}\b'.format(target_word), re.IGNORECASE)
        # if primary keyword is not found, match for 'tv'
        else:
            target_word_pattern = re.compile(r'\b{0}\b'.format(award.index('tv')), re.IGNORECASE)
                
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            weight = 10 if any('presenter' in tup for tup in match) else 1
            match_target_word = re.findall(target_word_pattern, line.lower())
            # if line contains at least number keywords to match and pattern is found
#             if match and len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match_target_word:
            if match and len(set(award).intersection(set(line.split()))) == num_keywords_to_match:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity = entity.strip() 
                    if len(entity) > 1:
                        if entity not in entity_dict:
                            entity_dict[entity] = weight
                        else:
                            entity_dict[entity] += weight
                num += 1
    
    else:
        for line in cleansed_data:
            match = re.findall(pattern, line.lower())
            weight = 10 if any('presenter' in tup for tup in match) else 1
            # if line contains at least number keywords to match and pattern is found
            if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match:
                tags = identify_entities(line)

                if verbose:
                    # print the first 5 occurrences
                    if num < 5:
                        print(match)
                        print(any('presenter' in tup for tup in match))
                        print(line)
                        print(tags.keys())
                        print()

                for entity in tags.keys():
                    entity = entity.strip() 
                    if len(entity) > 1:    
                        if entity not in entity_dict:
                            entity_dict[entity] = weight
                        else:
                            entity_dict[entity] += weight
                num += 1
    
    # if no results found, recursively reducing num_keywords_to_match
    while(num == 0):
        print('no results found!!! reducing num_keywords_to_match!!!')
        num_keywords_to_match -= 1
        if num_keywords_to_match == 0:
            break
            
        if flag == 1:       
#             print("matching target word '{0}'".format(target_word))
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                weight = 10 if any('presenter' in tup for tup in match) else 1
                match_target_word = re.findall(target_word_pattern, line.lower())
                # if line contains at least number keywords to match and pattern is found
#                 if match and len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match_target_word:
                if match and len(set(award).intersection(set(line.split()))) == num_keywords_to_match:
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity = entity.strip() 
                        if len(entity) > 1:    
                            if entity not in entity_dict:
                                entity_dict[entity] = weight
                            else:
                                entity_dict[entity] += weight
                    num += 1
        else:
            for line in cleansed_data:
                match = re.findall(pattern, line.lower())
                weight = 10 if any('presenter' in tup for tup in match) else 1
                # if line contains at least number keywords to match and pattern is found
                if len(set(award).intersection(set(line.split()))) == num_keywords_to_match and match:
                    tags = identify_entities(line)

                    if verbose:
                        # print the first 5 occurrences
                        if num < 5:
                            print(match)
                            print(line)
                            print(tags.keys())
                            print()

                    for entity in tags.keys():
                        entity = entity.strip() 
                        if len(entity) > 1:    
                            if entity not in entity_dict:
                                entity_dict[entity] = weight
                            else:
                                entity_dict[entity] += weight
                    num += 1
    print('num of matches:', num)
    return entity_dict

# Predictions for Award Presenters

In [70]:
%%time
for key in award_num_keywords_map.keys():
    entity_freq_dict = find_award_presenter(awards_reduced, key, verbose=False)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:100]
    # remove 'golden globes' from identified host names
    entity_freq_dict = remove_goldeb_globes(top_results, entity_freq_dict)
    top_results = sorted(entity_freq_dict.items(), key=lambda pair: pair[1], reverse=True)[:30]
    # filter for names if necessary
    top_results, entity_freq_dict = filter_names(top_results, entity_freq_dict)
    print('top results:')
    print(top_results)
    print('top results after merging:')
    top_10 = merge_names(top_results, entity_freq_dict)
    print(top_10)
    print()

0
Predicting for: best screenplay - motion picture
true presenter: ['robert pattinson', 'amanda seyfried']
award name reduced: ['screenplay']
num keywords to match: 1
num of matches: 6
top results:
[('Robert Pattinson', 3), ('Amanda Seyfried', 1), ("Robert Pattinson 's", 1)]
top results after merging:
[('Robert Pattinson', 4), ('Amanda Seyfried', 1)]

1
Predicting for: best director - motion picture
true presenter: ['halle berry']
award name reduced: ['director']
num keywords to match: 1
num of matches: 9
top results:
[('Ben Affleck', 32), ('Jennifer Garner', 30), ('Halle Berry', 5), ('El', 2)]
top results after merging:
[('Ben Affleck', 32), ('Jennifer Garner', 30), ('Halle Berry', 5), ('El', 2)]

2
Predicting for: best performance by an actress in a television series - comedy or musical
true presenter: ['aziz ansari', 'jason bateman']
award name reduced: ['actress', 'tv', 'series', 'comedy', 'musical']
num keywords to match: 4
no results found!!! reducing num_keywords_to_match!!!
num

no results found!!! reducing num_keywords_to_match!!!
num of matches: 3
top results:
[('Paul Rudd', 10), ('Dennis Quaid', 1), ('Kerry Washington', 1)]
top results after merging:
[('Paul Rudd', 10), ('Dennis Quaid', 1), ('Kerry Washington', 1)]

18
Predicting for: best performance by an actor in a mini-series or motion picture made for television
true presenter: ['jessica alba', 'kiefer sutherland']
award name reduced: ['actor', 'mini', 'series', 'tv']
num keywords to match: 3
num of matches: 1
top results:
[('Dennis Quaid', 1), ('Kerry Washington', 1)]
top results after merging:
[('Dennis Quaid', 1), ('Kerry Washington', 1)]

19
Predicting for: best performance by an actress in a mini-series or motion picture made for television
true presenter: ['don cheadle', 'eva longoria']
award name reduced: ['actress', 'mini', 'series', 'tv']
num keywords to match: 3
no results found!!! reducing num_keywords_to_match!!!
num of matches: 23
top results:
[('Maggie Smith', 21), ('Downton Abbey', 21), 